In [16]:
import torch
import numpy as np
import pandas as pd


data = pd.read_csv('data/TSLA.csv')
data = data.dropna()
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-06-29,19.000000,25.00,17.540001,23.889999,23.889999,18766300
1,2010-06-30,25.790001,30.42,23.299999,23.830000,23.830000,17187100
2,2010-07-01,25.000000,25.92,20.270000,21.959999,21.959999,8218800
3,2010-07-02,23.000000,23.10,18.709999,19.200001,19.200001,5139800
4,2010-07-06,20.000000,20.00,15.830000,16.110001,16.110001,6866900


In [17]:
data.drop(['Date'], axis=1, inplace=True)
means = data.mean(axis=0)
stds = data.std(axis=0)

data = (data-means) / stds
data.head()

,Open,High,Low,Close,Adj Close,Volume
0,-1.408716,-1.361362,-1.415198,-1.364102,-1.364102,2.645165
1,-1.351532,-1.316529,-1.365907,-1.364605,-1.364605,2.328553
2,-1.358185,-1.353752,-1.391836,-1.380302,-1.380302,0.530509
3,-1.375029,-1.377078,-1.405186,-1.403468,-1.403468,-0.086796
4,-1.400294,-1.402721,-1.429831,-1.429405,-1.429405,0.259468


In [18]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

ratio_trains = 0.8
trains = int ( len(data)* ratio_trains )

targets = torch.FloatTensor(data['Close'].values).to(device)
inputs = torch.FloatTensor(data[['Open', 'High', 'Low', 'Volume']].values).to(device)

targets_train = targets[:trains]
targets_test = targets[trains:]

inputs_train = inputs[:trains]
inputs_test = inputs[trains:]
print(inputs_train.shape, targets_train.shape, inputs_test.shape, targets_test.shape)

torch.Size([1932, 4]) torch.Size([1932]) torch.Size([484, 4]) torch.Size([484])


In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class CustomDataset(Dataset):
    def __init__(self,  df_file, ratio_trains, ds_type):
        self.df_file=df_file
        self.df=pd.read_csv(self.df_file)
        self.ratio_trains = ratio_trains
        self.ds_type=ds_type
        self.num_train=int(self.ratio_trains*len(self.df))
        self.num_test=len(self.df)-self.num_train
        if self.ds_type=='train':
            self.df=self.df.iloc[:self.num_train]
        if self.ds_type=='test':
            self.df=self.df.iloc[self.num_train:]

    def __len__(self):
        if self.ds_type=='train':
            self.df=self.num_train
        if self.ds_type=='test':
            self.df=self.num_test
        return len(self.inputs)

    def __getitem__(self, idx):  # 
        input = self.inputs[idx]
        target = self.targets[idx]        
        return torch.FloatTensor(input), torch.FloatTensor(target)
    
dataset_train = CustomDataset(inputs_train, targets_train) #tuple of inputs and targets
dataset_test = CustomDataset(inputs_test, targets_test) #tuple of inputs and targets

In [27]:
batch_size = 256
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)

# Batch Sampling
for input,target in train_loader:
    print(input.shape, target.shape)

torch.Size([256, 4]) torch.Size([256])
torch.Size([256, 4]) torch.Size([256])
torch.Size([256, 4]) torch.Size([256])
torch.Size([256, 4]) torch.Size([256])
torch.Size([256, 4]) torch.Size([256])
torch.Size([256, 4]) torch.Size([256])
torch.Size([256, 4]) torch.Size([256])
torch.Size([140, 4]) torch.Size([140])
